# Gibbs Sampling

This is useful for sampling from complex distribution $-$ multi-dimensional distribution  by component-wise sampling indenpendently. For example, we want to model three parameters, $\theta_{1}$, $\theta_{2}$ and $\theta_{3}$ sequentially as follows.

1. Initialize $\theta_{1}^{0}$, $\theta_{2}^{0}$ and $\theta_{3}^{0}$ as some value
2. for each iteration $i$:
 - create a new value $\theta_{1}^{i}$ conditioned on values $\theta_{2}^{i-1}$ and $\theta_{3}^{i-1}$
 - create a new value $\theta_{2}^{i}$ conditioned on values $\theta_{1}^{i}$ and $\theta_{3}^{i-1}$
 - create a new value $\theta_{3}^{i}$ conditioned on values $\theta_{1}^{i}$ and $\theta_{2}^{i}$

![example](https://donghwa-kim.github.io/picture/gibbs_seque.png)

# LDA with Gibbs Sampling

There is an approach to estimate the posterior distrubution over assigning words to topics, namely, $\theta$ and $\phi$ can be derived from $z$. For each word $i$, $z_{i}$ is an intenger value $[1,...,T]$ representing the topic a word is assiged to.

Let's assume a few notations:
- $d_{i}$ : $i^{th}$ document.
- $w_{d_{i}}$ : a word token in the document
and the topic assignment $z_{i}$ is estimated from $w_{d_{i}}$, conditioned on all other topic assignments to other word tokens.

![eq1](https://raw.githubusercontent.com/Donghwa-KIM/lda/master/figure/equ1.PNG)

where $\cdot$ means all other words $w_{i}$ and documents $d_{i}$ and $\propto$ is proportional notation, for example, $y\propto x \equiv y=kx$

$C^{wt}$ and $C^{DT}$ are matrics of counts with dimensions $W$ X $T$ and $D$ X $T$ respectively:
- $C^{WT}_{wj}$ is the count of words $w$ assigned to topic j, not including current instance $i$.
- $C^{DT}_{dj}$ is the count of topic j assigned to some word token in documnet D not including current instance i.

If many tokens of word $i$ have been assigned to a topic $j$, the probability that the following word token $i$ is assigned to the topic $j$ is increased according to frequency of the word. Similarly, if topic $j$ has been used multiple times within a document, it will increase the probability that any word within that document is assigned to topic $j$.

$\theta$ $-$ topic distribution and $\phi$ $-$ term distribution can be estimated by the following formula [M Steyvers & T Griffiths (2004)](https://www.dropbox.com/s/5v9lh0lcmx0nqfz/sciencetopics.pdf?dl=0) :

![eq2](https://raw.githubusercontent.com/Donghwa-KIM/lda/master/figure/eq2.PNG)

The gibbs sampling procedure now can be written as:
- Assign each word token $w_{i}$ a random topic[1,...,T]
- For each word token $w_{i}$:
    - Decrement count matrices $C^{WT}$ and $C^{DT}$ by one for current topic assignment.
    - Sample a new topic from the **_posterior equation_** ($\theta$, $\phi$) above.
    - Update count matrices $C^{WT}$ and $C^{DT}$ by one with the new sampled topic assignment.
- Repeat above steps until the iteration that you want as a hyperparameter.
- derive $\phi^{'}$ and $\theta^{'}$ from gibbs samples $z$, using the above equations.

there is an initial duration for **_'burn-in'_** where the poor estimates are often discared. After this duration, the samples start to approach the target distribution. Furthermore, to get a representative sample, samples are saved at regularly spaced intervals to prevent correlation between them.

# The process of LDA with an example using python

1) **_Data generation for an example_**

Suppose that there are two topics in $V$ = ['money', 'loan', 'bank', 'river', 'stream'] and to initialize, Topic 1 allocates equal probability to  'money', 'loan' and 'bank', i.e, $\phi^{1}_{money}=1/3$, $\phi^{1}_{loan}=1/3$, $\phi^{1}_{bank}=1/3$, whereas Topic 2 allocates equal probability to 'bank', 'river' and 'stream', i.e, $\phi^{2}_{bank}=1/3$, $\phi^{2}_{river}=1/3$, $\phi^{2}_{stream}=1/3$.

In [48]:
vocab = ["money",
        "loan",
        "bank",
        "river",
        "stream"]
vocab

['money', 'loan', 'bank', 'river', 'stream']

In [49]:
import numpy as np
z_1 = np.array([1/3, 1/3, 1/3, .0, .0])
z_2 = np.array([.0, .0,1/3, 1/3, 1/3])
z = np.array([z_1, z_2])
z = np.transpose(z)
z

array([[ 0.33333333,  0.        ],
       [ 0.33333333,  0.        ],
       [ 0.33333333,  0.33333333],
       [ 0.        ,  0.33333333],
       [ 0.        ,  0.33333333]])

In [50]:
# the number of documents
D = 16

# mean word length of documents
mean_length = 10

# sample a length for each document using Poisson
len_doc = np.random.poisson(mean_length, size=D)
len_doc # the number of words in a document

array([10,  8, 14, 17,  5, 15, 16,  8,  6, 14,  7, 12,  8, 14, 15, 14])

In [51]:
# the fixed number of topics
T = 2 # the number of topics
docs = []
orig_topics = []
for i in range(D): # D = 16
    z = np.random.randint(0, 2) # an generated random intenger between 0 and 1 corresponding to z_1 and z_2 respectively.
    if z == 0:
        #This is going to select the vocabs by the z probability depending on the size: the number of vocabs going to be selected,
        # and then that is changed as list type 
        words = np.random.choice(vocab, size=(len_doc[i]),p=z_1).tolist()# when 1st z (z==0), the probaility of z_1 is used for seleting words  
    else:
        words = np.random.choice(vocab, size=(len_doc[i]),p=z_2).tolist()# when 2nd z (z==1), the probaility of z_2 is used for seleting words  
    orig_topics.append(z) #[] <- put the topics into the empty list sequentially
    docs.append(words) #[] <- put the words into the empty list sequentially

In [52]:
orig_topics

[1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0]

In [53]:
for i in range(len(docs)):
    print('Document',i+1,'|'," ".join(docs[i]))

Document 1 | stream bank stream river river stream bank bank river stream
Document 2 | loan money money loan loan loan loan loan
Document 3 | bank loan money loan bank money money loan loan money money bank money money
Document 4 | loan bank money money loan bank bank loan bank loan loan loan bank bank money money bank
Document 5 | loan loan loan loan bank
Document 6 | stream river stream stream river stream stream stream stream stream river stream river stream bank
Document 7 | stream river stream river bank stream stream river bank bank bank river stream river bank river
Document 8 | money money money bank bank money money loan
Document 9 | loan bank loan money bank loan
Document 10 | river stream bank river river stream stream stream bank stream river stream river bank
Document 11 | bank river bank river river bank bank
Document 12 | bank loan bank money money money loan loan money money bank money
Document 13 | stream stream river stream bank stream stream river
Document 14 | bank 

2) **_Initialize the notations_**

we need to initialize the notations as below:
- $w_{i}$ = $i^{th}$ word token in a document
- $d_{i}$ = $i^{th}$ document
- $z_{i}$ = $i^{th}$ topic assignment for **_a word token randomly_**

In [55]:
w_i = []
i = []
d_i = []
z_i = []

counter = 0


# go through each document
# Enumerate returns index, list covered by tuple format
for doc_idx, doc in enumerate(docs):
    # go through each word in eac document
    for word_idx, word in enumerate(doc):
        # word index in vocab
        w_i.append(np.where(np.array(vocab)==word)[0][0])
        # counter
        i.append(counter)
        # document index in total document
        d_i.append(doc_idx)
        # initialize some topic assignment
        z_i.append(np.random.randint(0, T))
        counter += 1
        
# convert everything consisting of indices to numpy
w_i = np.array(w_i)
d_i = np.array(d_i)
z_i = np.array(z_i)

For example below, $(w_{1}, d_{1}, z_{1})$ means that $4^{th}$ word in vocab in $1^{th}$ document is assigned to $2^{th}$ topic initially.

In [56]:
print(w_i,', len:',len(w_i)) # vocab index (0~4)
print(d_i,', len:',len(d_i)) # document index (0~15)
print(z_i,', len:',len(z_i)) # topic index (0~1)
# Every notation above is corresponded each other.

[4 2 4 3 3 4 2 2 3 4 1 0 0 1 1 1 1 1 2 1 0 1 2 0 0 1 1 0 0 2 0 0 1 2 0 0 1
 2 2 1 2 1 1 1 2 2 0 0 2 1 1 1 1 2 4 3 4 4 3 4 4 4 4 4 3 4 3 4 2 4 3 4 3 2
 4 4 3 2 2 2 3 4 3 2 3 0 0 0 2 2 0 0 1 1 2 1 0 2 1 3 4 2 3 3 4 4 4 2 4 3 4
 3 2 2 3 2 3 3 2 2 2 1 2 0 0 0 1 1 0 0 2 0 4 4 3 4 2 4 4 3 2 2 4 2 3 4 3 2
 2 2 4 3 4 2 1 2 1 1 2 1 2 2 2 2 2 0 1 0 2 0 0 2 1 0 0 1 1 0 1 0 0 0 1] , len: 183
[ 0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  4
  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  8  8  8  8  8  8  9
  9  9  9  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 11 11 11 11 11
 11 11 11 11 11 11 11 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 13 13
 13 13 13 13 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 15 15 15 15 15 15
 15 15 15 15 15 15 15 15] , len: 183
[1 0 1 0 1 1 0 0 0 1 1 1 1 0 1 1 0 0 1 0 0 0 0 1

In [57]:
WT = np.zeros((len(vocab),T)) # create an empty WT matrix
DT = np.zeros((D,T)) # create an empty DT matrix
print(WT) # Words X Topics

[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]


In [58]:
print(DT) # Documents X Topics

[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]


In [59]:
for idx, word_ in enumerate(vocab): # go through each word 
    # for each topic, count the number of times 
    # it is assigned to a word
    topics = z_i[np.where(w_i == idx)] # take the topic assignments where the ith vocab match with w_i
    for t in range(T):
        WT[idx,t] = sum(topics == t) # put a total of each topic(column) into each vocab idx (row)

In [60]:
# This is the same as above except that w_i is changed to d_i

for idx, doc_ in enumerate(range(D)):
    # for each topic, count the number of times 
    # it is assigned to a document
    topics = z_i[np.where(d_i == idx)] # take the topic assignments where the ith document match with d_i
    for t in range(T):
        DT[idx,t] = sum(topics == t) # put a total of each topic(column) into each document idx (row)

It is necessary to save the intial matrices to compare the final result.

In [61]:
# distribution for each word
WT_orig = WT.copy()
WT_orig

array([[ 11.,  24.],
       [ 25.,  12.],
       [ 32.,  19.],
       [ 14.,  12.],
       [ 13.,  21.]])

In [62]:
# distribution for each document
DT_orig = DT.copy()
DT

array([[  5.,   5.],
       [  3.,   5.],
       [  7.,   7.],
       [  9.,   8.],
       [  4.,   1.],
       [  7.,   8.],
       [  6.,  10.],
       [  4.,   4.],
       [  2.,   4.],
       [  8.,   6.],
       [  3.,   4.],
       [  7.,   5.],
       [  3.,   5.],
       [  8.,   6.],
       [ 10.,   5.],
       [  9.,   5.]])

3) **_Learning Gibbs Sampling iteratively_**

Basically, It's going to learn each of the word tokens $w_{i}$ and update the topic assingments $z_{i}$

$\phi_{1}$ and $\phi_{2}$ will estimate and record topic assignments for each word

In [63]:
# a hyper parameter to make better performance but we have to consider the running time
iters = 100
phi_T = []
# when iteration 100, we should make the matrix as follows to record the estimates
for t in range(T):
    phi_T.append(np.zeros((len(vocab),iters)))

phi_T

[array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

The Dirichlet prior ($\alpha$, $\beta$) below are determinant hyper-parameters to decide the shape of topic distributions. for example

### when $\alpha$ = 1, learning iteratively

![alpha](http://donghwa-kim.github.io/picture/a_1.png)

### when $\alpha$ = 0.1, learning iteratively

![alpha](http://donghwa-kim.github.io/picture/a_01.png)

Generally, it is proven from [M Steyvers & T Griffiths (2007)](https://www.dropbox.com/s/tdsyqunr9qmcdlx/SteyversGriffiths.pdf?dl=0);[M Steyvers & T Griffiths (2004)](https://www.dropbox.com/s/5v9lh0lcmx0nqfz/sciencetopics.pdf?dl=0) that the reasonable values for the hyperparameters are experimentally

$\alpha$ = 50 $/$ The number of topics

$\beta$ = 0.01

![eq2](https://raw.githubusercontent.com/Donghwa-KIM/lda/master/figure/eq2.PNG)

In [64]:
# but we just apply the Dirichlet priors for an example
beta = 1. 
alpha = 1.

for step in range(iters):
    # for each word token 
    for current in i: # i  for all the duplicate words 
        # get document index and word index
        doc_idx = d_i[current] 
        w_idx = w_i[current]
                
        # decrease count matrices, it means to leave out the current topic assignment 
        DT[doc_idx,z_i[current]] -= 1
        WT[w_idx,z_i[current]] -= 1
        
        # calculate new assignment 
        # according to the formula
        prob_word =  (WT[w_idx,:] + beta) / (WT[:,:].sum(axis=0) + len(vocab)* beta) # referring to the equation (1) above
        prob_document = (DT[doc_idx,:] + alpha) / (DT[doc_idx,:].sum(axis=0) + T*alpha) ## referring to the equation (2) above
        prob = prob_word * prob_document # prob=[Topic1,Topic2]*[Topic1,Topic2]=[Topic1,Topic2]
        
        # choosing z_i according to the probabilities for each topic
        z_i[current] = np.random.choice(np.arange(T), 1, p=prob/prob.sum())[0]  # np.arange(T) = index for topics
                                                                                # prob/prob.sum() for nomalization to 1

        # return the origin matrix before leaving out the current topic assignment 
        DT[doc_idx,z_i[current]] += 1
        WT[w_idx,z_i[current]] += 1

        # estimate phi - not essential for inference
        phi  = WT/(WT.sum(axis=0)) # generated topic assignments
        for t in range(T):
            phi_T[t][:,step] = phi[:,t] # record the generated topic assignments to topic 1
                                        # record the generated topic assignments to topic 2
    if((step+1) % 10 ==0):
        print('iteration: ', step+1)

iteration:  10
iteration:  20
iteration:  30
iteration:  40
iteration:  50
iteration:  60
iteration:  70
iteration:  80
iteration:  90
iteration:  100


In [65]:
DT # topic assingments for document X topic

array([[  0.,  10.],
       [  8.,   0.],
       [ 14.,   0.],
       [ 17.,   0.],
       [  5.,   0.],
       [  0.,  15.],
       [  0.,  16.],
       [  8.,   0.],
       [  6.,   0.],
       [  0.,  14.],
       [  0.,   7.],
       [ 11.,   1.],
       [  0.,   8.],
       [  0.,  14.],
       [ 13.,   2.],
       [ 14.,   0.]])

In [66]:
theta=DT/(DT.sum(axis=1)).reshape(DT.shape[0],1)# topic distribution over each document
theta

array([[ 0.        ,  1.        ],
       [ 1.        ,  0.        ],
       [ 1.        ,  0.        ],
       [ 1.        ,  0.        ],
       [ 1.        ,  0.        ],
       [ 0.        ,  1.        ],
       [ 0.        ,  1.        ],
       [ 1.        ,  0.        ],
       [ 1.        ,  0.        ],
       [ 0.        ,  1.        ],
       [ 0.        ,  1.        ],
       [ 0.91666667,  0.08333333],
       [ 0.        ,  1.        ],
       [ 0.        ,  1.        ],
       [ 0.86666667,  0.13333333],
       [ 1.        ,  0.        ]])

In [67]:
WT # topic assingments for word X topic

array([[ 35.,   0.],
       [ 37.,   0.],
       [ 24.,  27.],
       [  0.,  26.],
       [  0.,  34.]])

In [68]:
phi=WT/WT.sum(axis=0) # word distribution over each topic
phi

array([[ 0.36458333,  0.        ],
       [ 0.38541667,  0.        ],
       [ 0.25      ,  0.31034483],
       [ 0.        ,  0.29885057],
       [ 0.        ,  0.3908046 ]])

In [72]:
np.argmax(theta, axis=1) == orig_topics # all topic assingments were well predictied

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [74]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, axs = plt.subplots(T,1, sharey=False) # T row X 1 col


for t in range(T):
    for i in range(phi_T[t].shape[0]):
            axs[t].plot(phi_T[t][i,0:100]) # each word distribution over topic at i iteration
            axs[t].set_title('Topic '+ str(t+1))


fig.set_size_inches((7, 4)) # width, length
sns.despine()

for ax in axs:
    ax.set_ylabel(r'$\phi_w$')
    ax.set_xlabel('Iteration')
    ax.set_yticks(np.arange(0, 0.5, 0.1))
    
fig.tight_layout()
fig

Each panel shows the fluctaion a probability of assigning each word to topic j. 

Maybe later I am going to apply this conception using Database.
